In [8]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [9]:
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt")

time.sleep(3)

driver.maximize_window()

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

#implicit wait
wait = WebDriverWait(driver, 20)

def link_route(path):   
    LINKS = []
    ROUTE = []

    # Retrieve the route links and route names
    for i in range(1, 6):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS, ROUTE

# Calling the function to get links and routes
LINKS, ROUTE = link_route("//a[@class='route']")


No more pages to paginate at step 1


In [11]:
df_k=pd.DataFrame({"Route_name":ROUTE,"Route_link":LINKS})
df_k

,Route_name,Route_link
0,Siliguri to Gangtok,https://www.redbus.in/bus-tickets/siliguri-to-...
1,Gangtok to Siliguri,https://www.redbus.in/bus-tickets/gangtok-to-s...
2,Rangpo to Siliguri,https://www.redbus.in/bus-tickets/rangpo-to-si...
3,Siliguri to Rangpo,https://www.redbus.in/bus-tickets/siliguri-to-...
4,Singtham (Sikkim) to Siliguri,https://www.redbus.in/bus-tickets/singtham-sik...
5,Siliguri to Singtham (Sikkim),https://www.redbus.in/bus-tickets/siliguri-to-...
6,Ravangla (Sikkim) to Siliguri,https://www.redbus.in/bus-tickets/ravangla-sik...
7,Siliguri to Pelling (Sikkim),https://www.redbus.in/bus-tickets/siliguri-to-...


In [12]:
df = df_k

In [13]:
#retrive the bus details
driver_x = webdriver.Chrome()
Bus_names_x = []
Bus_types_x = []
Start_Time_x = []
End_Time_x = []
Ratings_x = []
Total_Duration_x = []
Prices_x = []
Seats_Available_x = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_x.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_x.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_x.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_x.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_x).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_x.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_x.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_x.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_x.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_x.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_x.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_x.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_x.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_x.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_x.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_x.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_x.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_x.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_x.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_x.append(ratings.text)
    for price_elem in price:
        Prices_x.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_x.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [14]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_x,
    'Bus_type': Bus_types_x,
    'Start_time': Start_Time_x,
    'End_time': End_Time_x,
    'Total_duration': Total_Duration_x,
    'Price': Prices_x,
    "Seats_Available":Seats_Available_x,
    "Ratings":Ratings_x,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)



In [15]:
#convert dataframe to csv
df1.to_csv("E:/snt.csv")